#### 비트 코인과 금 : 경제적 불확실성 시기에 안전자산으로서의 역할과 대체 가능성 분석

- 비트코인과 금은 경제적 불확실성이 클 때, 어떠한 양상을 보이는지? 
    - 금 가격과 비트코인 변동성 비교 / 경제 위기의 기간 동안의 트랜드 분석
    - 비트코인과 금은 가격 상승 및 상관관계를 보이는가?
    - 비트코인의 변동성은 금보다 크지만, 경제 위기 시 비트코인이 금과 유사한 안전자산의 역할을 수행할 수 있는가?


In [3]:
# !pip install kagglehub

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub

In [15]:
# Data kaggle API 

gold_path = kagglehub.dataset_download("farzadnekouei/gold-price-10-years-20132023")
print("Path to dataset files:", gold_path)

btc_path = kagglehub.dataset_download("prasoonkottarathil/btcinusd")
print("Path to dataset files:", btc_path)

Path to dataset files: C:\Users\Playdata\.cache\kagglehub\datasets\farzadnekouei\gold-price-10-years-20132023\versions\1


100%|██████████| 80.2M/80.2M [00:03<00:00, 23.7MB/s]

Extracting files...


Path to dataset files: C:\Users\Playdata\.cache\kagglehub\datasets\prasoonkottarathil\btcinusd\versions\4


In [85]:
# Gold Data Read

gold_data = pd.read_csv('./data/Gold Price (2013-2023).csv')
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data.info()
gold_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2583 entries, 0 to 2582
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2583 non-null   datetime64[ns]
 1   Price     2583 non-null   object        
 2   Open      2583 non-null   object        
 3   High      2583 non-null   object        
 4   Low       2583 non-null   object        
 5   Vol.      2578 non-null   object        
 6   Change %  2583 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 141.4+ KB


,Date,Price,Open,High,Low,Vol.,Change %
0,2022-12-30,"1,826.20","1,821.80","1,832.40","1,819.80",107.50K,0.01%
1,2022-12-29,"1,826.00","1,812.30","1,827.30","1,811.20",105.99K,0.56%
2,2022-12-28,"1,815.80","1,822.40","1,822.80","1,804.20",118.08K,-0.40%
3,2022-12-27,"1,823.10","1,808.20","1,841.90","1,808.00",159.62K,0.74%
4,2022-12-26,"1,809.70","1,805.80","1,811.95","1,805.55",NaN,0.30%
...,...,...,...,...,...,...,...
2578,2013-01-08,"1,663.20","1,651.50","1,662.60","1,648.80",0.13K,0.97%
2579,2013-01-07,"1,647.20","1,657.30","1,663.80","1,645.30",0.09K,-0.16%
2580,2013-01-04,"1,649.90","1,664.40","1,664.40","1,630.00",0.31K,-1.53%
2581,2013-01-03,"1,675.60","1,688.00","1,689.30","1,664.30",0.19K,-0.85%


In [91]:
def clean_and_convert(df):
    for col in ['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']:
        if df[col].dtype == 'O':
            if col == 'Vol.':
                df[col] = df[col].replace({'K': '*1e3', 'M': '*1e6'}, regex=True)
                df[col] = df[col].apply(lambda x: eval(str(x)) if isinstance(x, str) else x)
            elif col == 'Change %':
                df[col] = df[col].str.replace('%', '', regex=True) 
            df[col] = df[col].str.replace(',', '', regex=True) 
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

gold_data = clean_and_convert(gold_data)

In [92]:
gold_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2583 entries, 0 to 2582
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2583 non-null   datetime64[ns]
 1   Price     2583 non-null   float64       
 2   Open      2583 non-null   float64       
 3   High      2583 non-null   float64       
 4   Low       2583 non-null   float64       
 5   Vol.      2578 non-null   float64       
 6   Change %  2583 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 141.4 KB


In [93]:
gold_data.describe()

,Date,Price,Open,High,Low,Vol.,Change %
count,2583,2583.000000,2583.000000,2583.000000,2583.000000,2578.000000,2583.000000
mean,2018-01-02 02:17:08.571428608,1440.326442,1440.646477,1449.639934,1430.886876,94046.799845,0.008122
min,2013-01-02 00:00:00,1049.600000,1051.500000,1060.100000,1045.400000,0.000000,-9.340000
25%,2015-07-02 12:00:00,1244.200000,1244.350000,1251.600000,1236.450000,290.000000,-0.440000
50%,2018-01-03 00:00:00,1320.500000,1320.700000,1326.500000,1313.200000,7215.000000,0.010000
75%,2020-07-04 12:00:00,1713.950000,1711.950000,1723.000000,1699.450000,174342.500000,0.490000
max,2022-12-30 00:00:00,2058.400000,2065.100000,2078.700000,2037.200000,700340.000000,5.970000
std,NaN,257.338282,257.503397,259.633923,254.753279,117354.385623,0.975231


In [53]:
# BTC Data Read

btc_daily = pd.read_csv('./data/BTC-Daily.csv')
btc_daily['date'] = pd.to_datetime(btc_daily['date'])
btc_daily.info()
btc_daily.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651 entries, 0 to 2650
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   unix        2651 non-null   int64         
 1   date        2651 non-null   datetime64[ns]
 2   symbol      2651 non-null   object        
 3   open        2651 non-null   float64       
 4   high        2651 non-null   float64       
 5   low         2651 non-null   float64       
 6   close       2651 non-null   float64       
 7   Volume BTC  2651 non-null   float64       
 8   Volume USD  2651 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 186.5+ KB


,unix,date,open,high,low,close,Volume BTC,Volume USD
count,2.651000e+03,2651,2651.000000,2651.000000,2651.000000,2651.000000,2.651000e+03,2.651000e+03
mean,1.531613e+09,2018-07-15 00:00:00,11693.189332,12038.562912,11296.539461,11709.326213,1.330041e+07,6.099369e+07
min,1.417133e+09,2014-11-28 00:00:00,162.000000,212.840000,152.400000,162.000000,0.000000e+00,0.000000e+00
25%,1.474373e+09,2016-09-20 12:00:00,654.680000,664.460000,640.050000,654.370000,5.357883e+03,9.516170e+03
50%,1.531613e+09,2018-07-15 00:00:00,6408.950000,6540.000000,6312.000000,6407.770000,1.503310e+04,2.288982e+07
75%,1.588853e+09,2020-05-07 12:00:00,10708.620000,10956.240000,10303.755000,10726.425000,3.435336e+06,8.018383e+07
max,1.646093e+09,2022-03-01 00:00:00,67547.490000,69000.000000,66250.000000,67559.000000,7.723295e+08,1.446125e+09
std,6.613247e+07,NaN,16272.554746,16739.237000,15718.297511,16282.908704,4.715366e+07,1.072838e+08


In [94]:
btc_daily

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01,BTC/USD,43221.71,43626.49,43185.48,43185.48,4.900629e+01,2.116360e+06
1,1646006400,2022-02-28,BTC/USD,37717.10,44256.08,37468.99,43178.98,3.160618e+03,1.364723e+08
2,1645920000,2022-02-27,BTC/USD,39146.66,39886.92,37015.74,37712.68,1.701817e+03,6.418008e+07
3,1645833600,2022-02-26,BTC/USD,39242.64,40330.99,38600.00,39146.66,9.127241e+02,3.573010e+07
4,1645747200,2022-02-25,BTC/USD,38360.93,39727.97,38027.61,39231.64,2.202852e+03,8.642149e+07
...,...,...,...,...,...,...,...,...,...
2646,1417478400,2014-12-02,BTC/USD,378.39,382.86,375.23,379.25,2.593576e+06,6.832530e+03
2647,1417392000,2014-12-01,BTC/USD,376.40,382.31,373.03,378.39,2.520662e+06,6.660560e+03
2648,1417305600,2014-11-30,BTC/USD,376.57,381.99,373.32,373.34,1.145567e+06,3.046330e+03
2649,1417219200,2014-11-29,BTC/USD,376.42,386.60,372.25,376.72,2.746157e+06,7.245190e+03


- 교차 상관 분석과 자산군 간의 시차(Lag)를 활용하여 투자 신호 탐색 
    - 금 가격 상승이 비트 코인 상승보다 선행하여 이를 통해 투자 타이밍을 잡을 수 있음